In [1]:
import gct,os

In [2]:
names = [u for u in gct.list_local_graph() if u.startswith("snap_rand_N")]
len(names)

50

In [3]:
names[:2]

['snap_rand_N32', 'snap_rand_N38']

In [4]:
!pip install func_timeout

You are using pip version 9.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
def run_alg(algname):
    for dataname in names:
        print("running", algname,dataname)
        retcode = os.system("timeout -k 1h 1h python run_alg.py {} {} 0 > /data/log/{}_{}.log 2>&1".format(algname,dataname,algname,dataname))
        if retcode !=0:
            break


In [6]:
import numpy as np
algs =gct.list_algorithms()
np.random.shuffle(algs)
len(algs)

79

In [7]:
from joblib import Parallel, delayed
ret=Parallel(n_jobs=6)(delayed(run_alg)(x) for x in algs)    

In [15]:
run_alg('alg_GossipMap')

running alg_GossipMap snap_rand_N32
running alg_GossipMap snap_rand_N38
running alg_GossipMap snap_rand_N45
running alg_GossipMap snap_rand_N54
running alg_GossipMap snap_rand_N64
running alg_GossipMap snap_rand_N76
running alg_GossipMap snap_rand_N91
running alg_GossipMap snap_rand_N109
running alg_GossipMap snap_rand_N130
running alg_GossipMap snap_rand_N156
running alg_GossipMap snap_rand_N187
running alg_GossipMap snap_rand_N224
running alg_GossipMap snap_rand_N268
running alg_GossipMap snap_rand_N321
running alg_GossipMap snap_rand_N385
running alg_GossipMap snap_rand_N462
running alg_GossipMap snap_rand_N554
running alg_GossipMap snap_rand_N664
running alg_GossipMap snap_rand_N796
running alg_GossipMap snap_rand_N955
running alg_GossipMap snap_rand_N1146
running alg_GossipMap snap_rand_N1375
running alg_GossipMap snap_rand_N1650
running alg_GossipMap snap_rand_N1980
running alg_GossipMap snap_rand_N2376
running alg_GossipMap snap_rand_N2851
running alg_GossipMap snap_rand_N3421
r

In [18]:
run_alg('mcl_MCL')

running mcl_MCL snap_rand_N32
running mcl_MCL snap_rand_N38
running mcl_MCL snap_rand_N45
running mcl_MCL snap_rand_N54
running mcl_MCL snap_rand_N64
running mcl_MCL snap_rand_N76
running mcl_MCL snap_rand_N91
running mcl_MCL snap_rand_N109
running mcl_MCL snap_rand_N130
running mcl_MCL snap_rand_N156
running mcl_MCL snap_rand_N187
running mcl_MCL snap_rand_N224
running mcl_MCL snap_rand_N268
running mcl_MCL snap_rand_N321
running mcl_MCL snap_rand_N385
running mcl_MCL snap_rand_N462
running mcl_MCL snap_rand_N554
running mcl_MCL snap_rand_N664
running mcl_MCL snap_rand_N796
running mcl_MCL snap_rand_N955
running mcl_MCL snap_rand_N1146
running mcl_MCL snap_rand_N1375
running mcl_MCL snap_rand_N1650
running mcl_MCL snap_rand_N1980
running mcl_MCL snap_rand_N2376
running mcl_MCL snap_rand_N2851
running mcl_MCL snap_rand_N3421
running mcl_MCL snap_rand_N4105
running mcl_MCL snap_rand_N4926
running mcl_MCL snap_rand_N5911
running mcl_MCL snap_rand_N7093
running mcl_MCL snap_rand_N8511
run

In [ ]:
run_alg('igraph_community_optimal_modularity')

running igraph_community_optimal_modularity snap_rand_N32


In [9]:
import numpy as np
np.where(not ret), len(ret)

((array([], dtype=int64),), 9)

In [4]:
results = {}
d = gct.list_all_clustering_results(print_format=False)
for u in d:
    if "snap_rand_N" in u:
        for run in d[u]:
            a = gct.load_clustering_result(u, run)
            results[u + run] = a

len(results)

3591

In [5]:
import pandas as pd
df = pd.DataFrame([(u['runname'],u['dataname'],u['timecost']) for u in results.values()], columns=['alg','datanem','timecost'])

In [6]:
missedargs = set(gct.list_algorithms()).difference(df['alg'].values)
missedargs


{'igraph_community_optimal_modularity'}

In [7]:
with pd.option_context('display.max_rows', 2000, 'display.max_columns', 200):
    display (df.sort_values("timecost"))

,alg,datanem,timecost
64,igraph_community_label_propagation,snap_rand_N32,0.000109
141,igraph_community_label_propagation,snap_rand_N38,0.000131
217,igraph_community_label_propagation,snap_rand_N45,0.000142
295,igraph_community_label_propagation,snap_rand_N54,0.000162
373,igraph_community_label_propagation,snap_rand_N64,0.000165
...,...,...,...
1619,igraph_community_edge_betweenness,snap_rand_N1146,3400.226703
3024,cdc_clique_modularity,snap_rand_N43909,3420.780696
1392,snap_Girvan_Newman,snap_rand_N664,3498.884099
2511,cdc_EAGLE,snap_rand_N10213,3564.237827


In [8]:
df['N']=df['datanem'].map(lambda u: int(u.split("_")[-1][1:]))

In [9]:
with pd.option_context('display.max_rows', 2000, 'display.max_columns', 200):
    pdf=df.pivot_table(columns=['N'], index=['alg'], values='timecost')
    for x in missedargs:
        pdf=pdf.append(pd.Series(name=x))

    display(pdf)

/opt/conda/envs/python3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


,32,38,45,54,64,76,91,109,130,156,187,224,268,321,385,462,554,664,796,955,1146,1375,1650,1980,2376,2851,3421,4105,4926,5911,7093,8511,10213,12255,14706,17647,21176,25411,30493,36591,43909,52690,63228,75873,91047,109256,131107,157328,188793,226551
alg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
alg_GossipMap,0.870206,0.696503,0.687716,0.700482,0.671058,0.684356,0.675031,0.677113,0.678720,0.685801,0.697027,0.692453,0.690298,0.695169,0.725501,0.730475,0.861109,0.897863,1.028521,1.067350,1.276537,1.389904,1.527275,1.802849,1.687650,2.029963,2.129894,2.952074,3.421204,4.554248,4.557168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
alg_Paris,0.010316,0.014698,0.019040,0.019607,0.013987,0.016799,0.035975,0.045540,0.055681,0.067694,0.082617,0.112322,0.134822,0.154107,0.205734,0.249539,0.355271,0.411805,0.454444,0.618610,0.843633,1.044094,1.157292,1.367065,1.829483,2.222694,1.572466,4.469666,5.116152,5.457280,6.813446,8.014348,10.047489,14.919797,18.749886,22.989770,16.628428,35.907974,30.225846,46.479297,55.585729,82.545420,101.430195,71.336138,76.945311,97.043292,184.893687,234.932078,273.487186,337.662992
alg_RelaxMap,0.019302,0.016242,0.016749,0.016352,0.015888,0.017596,0.017350,0.019010,0.019925,0.020430,0.028864,0.030746,0.029630,0.046099,0.057633,0.139496,0.096348,0.137061,0.186899,0.220270,0.387913,0.308091,0.445504,0.549054,0.759657,0.906067,1.188642,0.918733,1.587297,1.877792,2.256095,2.635522,2.925293,4.090237,5.513632,5.305522,8.584558,9.701276,10.356201,13.733357,17.917637,22.765596,24.759773,30.288201,33.351102,47.031307,56.462493,52.022935,83.599824,96.519099
alg_lso_cluster,0.018021,0.011351,0.011666,0.015863,0.016471,0.016347,0.016652,0.017423,0.014408,0.019718,0.016630,0.021987,0.022264,0.025443,0.025948,0.030204,0.033837,0.038609,0.047478,0.052879,0.064971,0.068762,0.080091,0.104608,0.117517,0.137541,0.190582,0.240102,0.259259,0.312812,0.415655,0.482450,0.651107,0.799927,0.930576,1.376700,1.430132,2.060759,2.109465,2.962165,3.743330,4.802703,5.839899,7.408892,8.228130,10.473597,12.782325,18.085739,21.767906,29.394736
alg_pg_label_propagation,0.673945,0.409294,0.411172,0.407346,0.406003,0.411176,0.410254,0.407542,0.410265,0.411973,0.404963,0.414495,0.416671,0.408670,0.406363,0.415965,0.415070,0.415519,0.417604,0.418620,0.426416,0.423885,0.432140,0.431297,0.444310,0.444687,0.453788,0.474318,0.502089,0.542879,0.586863,0.633898,0.680333,0.767294,0.868980,0.977856,1.101182,1.248846,1.576003,1.669959,2.168378,2.439151,2.902682,3.601927,4.125959,4.995009,5.906734,7.715461,8.546815,10.717439
alg_streamcom,0.014042,NaN,0.014961,NaN,0.011085,0.013031,0.012965,0.011155,NaN,0.011057,0.011623,0.010045,0.016949,0.015870,0.015584,NaN,NaN,0.015171,0.017849,0.015795,NaN,0.019639,NaN,0.021968,0.023059,0.025267,0.025539,0.024063,NaN,0.032241,0.039173,0.044359,0.051197,0.062515,NaN,0.091783,0.098936,0.074170,0.141366,0.178878,0.214030,NaN,0.296234,0.392924,0.532817,0.551807,0.682146,0.853922,1.023842,1.295228
cdc_CONGA,0.130448,0.134142,0.147526,0.145843,0.151682,0.169819,0.168253,0.173235,0.183551,0.178268,0.190032,0.186674,0.225119,0.254812,0.259893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cdc_CliquePercolation,0.009519,0.009306,0.013634,0.009756,0.013797,0.014210,0.015136,0.014767,0.015235,0.017493,0.011435,0.020473,0.020952,0.021008,0.025179,0.027499,0.029633,0.019930,0.034957,0.040132,0.045651,0.047873,0.056846,0.069517,0.074409,0.085876,0.096752,0.120930,0.145304,0.168658,0.208160,0.259844,0.271241,0.322905,0.449818,0.533202,0.644448,0.782809,0.871244,1.117318,1.318459,1.634042,1.423468,2.236726,3.084917,3.813032,4.763772,5.706230,6.687510,9.114450
cdc_Connected_Iterative_Scan,0.028527,0.030881,0.039006,0.045798,0.063745,0.086030,0.114698,0.164898,0.237696,0.359951,0.512530,0.760225,1.124940,1.725692,2.868812,4.183916,5.378874,8.451843,12.073707,16.220499,23.329707,36.648809,58.401866,85.011197,124

In [58]:
pd.Series(name=x)

Series([], Name: snap_Girvan_Newman, dtype: float64)

In [56]:
pdf.shape

(61, 50)

In [35]:
pdf.max(axis=1)

alg
alg_RelaxMap                    114.660106
alg_lso_cluster                  36.857927
alg_pg_label_propagation          0.579085
alg_streamcom                     1.627761
cdc_CONGA                         0.280643
                                  ...     
scan_pScan                        0.597787
scan_ppScan                       0.190575
scan_ppScanSSE                    0.194713
sklearn_AffinityPropagation    2586.744437
sklearn_SpectralClustering     3053.078523
Length: 61, dtype: float64

In [36]:
os.system("ls > /tmp/a")

0

In [16]:
def run_alg(row):
    for N in row.index:
        algname=row.name
        t=row[N]
        if t>2000:
            break
        if np.isnan(t):
            dataname='snap_rand_N'+str(N)
            print("running", algname,dataname)
            if 1:
                retcode = os.system("timeout -k 1h 1h python run_alg.py {} {} 0 > /data/log/{}_{}.log 2>&1".format(algname,dataname,algname,dataname))
                if retcode !=0:
                    break


In [17]:
for i in pdf.index:
    #run_alg(pdf.loc[i])
    pass

In [18]:
args=[pdf.loc[i] for i in pdf.index]
len(args)

79

In [74]:

from joblib import Parallel, delayed
ret=Parallel(n_jobs=6)(delayed(run_alg)(x) for x in args)    

In [19]:
run_alg(pdf.loc['cdc_HDEMON'])

running cdc_HDEMON snap_rand_N32
running cdc_HDEMON snap_rand_N38
running cdc_HDEMON snap_rand_N45
running cdc_HDEMON snap_rand_N54
running cdc_HDEMON snap_rand_N64
running cdc_HDEMON snap_rand_N76
running cdc_HDEMON snap_rand_N91
running cdc_HDEMON snap_rand_N109
running cdc_HDEMON snap_rand_N130
running cdc_HDEMON snap_rand_N156
running cdc_HDEMON snap_rand_N187
running cdc_HDEMON snap_rand_N224
running cdc_HDEMON snap_rand_N268
running cdc_HDEMON snap_rand_N321
running cdc_HDEMON snap_rand_N385
running cdc_HDEMON snap_rand_N462
running cdc_HDEMON snap_rand_N554
running cdc_HDEMON snap_rand_N664
running cdc_HDEMON snap_rand_N796
running cdc_HDEMON snap_rand_N955
running cdc_HDEMON snap_rand_N1146
running cdc_HDEMON snap_rand_N1375
running cdc_HDEMON snap_rand_N1650
running cdc_HDEMON snap_rand_N1980
running cdc_HDEMON snap_rand_N2376
running cdc_HDEMON snap_rand_N2851
running cdc_HDEMON snap_rand_N3421
running cdc_HDEMON snap_rand_N4105
running cdc_HDEMON snap_rand_N4926
running cdc_